In [6]:
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import pickle

全连接层中 数据的形状被忽视了

卷积层的输入输出数据称为特征图(feature map)      

(height,width)

填充主要是为了调整输出的大小

OH = (H+2P-FH)/S + 1  OW = (W+2P-FW)/S + 1

输入数据的通道数要和卷积核通道数一样 一个卷积核运算后得到的是单通道的特征图 所以有多少个卷积核输出结果通道数就为多少

池化层 缩小height width

一般来说 池化的窗口大小会和步幅设定成相同的值

1.没有要学习的参数  2.通道数不发生变化  3.对微小的位置变化具有健壮性

7.4 卷积层和池化层的实现

In [7]:
#7.4.1 4维数组
x = np.random.rand(10,1,28,28)
x.shape,x[0].shape,x[1].shape

((10, 1, 28, 28), (1, 28, 28), (1, 28, 28))

In [8]:
#im2col
#对三维数据展开以适合卷积核 这里是对包含批数量的4维数据转换为二维数据
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """

    Parameters
    ----------
    input_data : 由(数据量, 通道, 高, 长)的4维数组构成的输入数据
    filter_h : 滤波器的高
    filter_w : 滤波器的长
    stride : 步幅
    pad : 填充

    Returns
    -------
    col : 2维数组
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """

    Parameters
    ----------
    col :
    input_shape : 输入数据的形状（例：(10, 1, 28, 28)）
    filter_h :
    filter_w
    stride
    pad

    Returns
    -------

    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

卷积层

初始化方法是把卷积核、偏置、不付、填充作为参数接收

In [9]:
class Convolution:
    def __init__(self,w,b,stride = 1,pad = 0) -> None:
        self.w = w
        self.b = b
        self.stride = stride
        self.pad = pad

        #中间数据 backward时使用
        self.x = None
        self.col = None
        self.col_w = None

        #权重和偏置参数的梯度
        self.dw = None
        self.db = None

    def forward(self,x):
        FN,C,FH,FW =self.w.shape
        N,C,W,H = x.shape
        out_h = int((H+2*self.pad - FH)/self.stride + 1)
        out_w = int((W+2*self.pad - FW)/self.stride + 1)

        col = im2col(x,FH,FW,self.stride,self.pad)
        col_w = self.w.reshape(FN,-1).T
        out = np.dot(col,col_w)+self.b

        #第一个N并不是输出特征图的通道数 而是批处理代表的N个结果 最后-1自动计算的结果为输出特征图的通道数
        out = out.reshape(N,out_h,out_w,-1).transpose(0,3,1,2)
        #N H W C --> N C H W

        self.x = x
        self.col = col
        self.col_w = col_w

        return out
    
    def backward(self,dout):
        FN,C,FH,FW = self.w.shape

        #N C H W --> N H W C
        dout = dout.transpose(0,2,3,1).reshape(-1,FN)

        self.db = np.sum(dout,axis=0)
        self.dw = np.dot(self.col.T,dout)
        self.dw = self.dw.transpose(1,0).reshape(FN,C,FH,FW)

        dcol = np.dot(dout,self.col_w.T)
        dx = col2im(dcol,self.x.shape,FH,FW,self.stride,self.pad)

        return dx


池化层

池化层展开的时候在通道方向上是独立的 

In [10]:
class pooling:
    def __init__(self,pool_h,pool_w,stride = 1,pad = 0) -> None:
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

    def forward(self,x):
        N,C,H,W = x.shape
        out_h = int(1+(H - self.pool_h)/self.stride)
        out_w = int(1+(W - self.pool_w)/self.stride)

        # 1.展开各行数据


        col = im2col(x,self.pool_h,self.pool_w,self.stride,self.pad)
        col = col.reshape(-1,self.pool_h*self.pool_w)
        # 2.求各行最大值
        out = np.max(col,axis=1)
        # 3.转换为合适的输出大小
        out = out.reshape(N,out_h,out_w,C).transpose(0,3,1,2)

        return out
    
    
    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx


In [11]:
class Relu:
    def __init__(self) -> None:
        self.mask = None

    def forward(self,x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0

        return out
    
    def backward(self,dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [12]:
class Affine:
    def __init__(self,w,b) -> None:
        self.w = w
        self.b = b
        self.x = None
        self.original_x_shape = None
        self.dw = None
        self.db = None

    def forward(self,x):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0],-1)
        self.x = x
        out = np.dot(self.x,self.w)+self.b

        return out
    
    def backward(self,dout):

        dx = np.dot(dout,self.w.T)
        self.dw = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis=0)

        dx = dx.reshape(*self.original_x_shape)
        return dx

In [13]:
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x,axis=0)
        y = np.exp(x)/np.sum(np.exp(x),axis=0)
        return y.T
    
    x = x - np.max(x)
    return np.exp(x)/np.sum(np.exp(x))

In [14]:
def cross_entropy_error(y,t):
    if y.ndim == 1:
        y = y.reshape(1,y.size)
        t = t.reshape(1,t.size)

    if t.size == y.size:
        t = t.argmax(axis=1)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size),t]+1e-7))/batch_size

In [15]:
class SoftmaxWithLoss:
    def __init__(self) -> None:
        self.loss = None
        self.y = None
        self.t = None

    def forward(self,x,t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t)

        return self.loss
    
    def backward(self,dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size:
            dx = (self.y - self.t)/batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx



In [16]:
class SimpleConvNet:
    def __init__(self,input_dim=(1,28,28),
                 conv_param={'filter_num':30,'filter_size':5,'pad':0,'stride':1},
                 hidden_size = 100,output_size = 10,weight_init_std = 0.01) -> None:
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']

        input_size = input_dim[1]

        conv_output_size = (input_size - filter_size +2*filter_pad)/filter_stride + 1
        pool_output_size = int(filter_num*(conv_output_size/2)*(conv_output_size/2))

        self.params = {}

        self.params['w1'] = weight_init_std*np.random.rand(filter_num,input_dim[0],filter_size,filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['w2'] = weight_init_std*np.random.randn(pool_output_size,hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['w3'] = weight_init_std*np.random.rand(hidden_size,output_size)
        self.params['b3'] = np.zeros(output_size)

        self.layers = OrderedDict()
        self.layers['conv1'] = Convolution(self.params['w1'],
                                           self.params['b1'],
                                           conv_param['stride'],
                                           conv_param['pad'])
        
        self.layers['relu1'] = Relu()
        self.layers['pool1'] = pooling(pool_h=2,pool_w=2,stride=2)
        self.layers['affine1'] = Affine(self.params['w2'],
                                        self.params['b2'])
        
        self.layers['affine2'] = Affine(self.params['w3'],
                                        self.params['b3'])
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self,x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self,x,t):
        y = self.predict(x)
        loss = self.last_layer.forward(y,t)

    def gradient(self,x,t):
        

        self.loss(x,t)

        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        grads['w1'], grads['b1'] = self.layers['conv1'].dw, self.layers['conv1'].db
        grads['w2'], grads['b2'] = self.layers['affine1'].dw, self.layers['affine1'].db
        grads['w3'], grads['b3'] = self.layers['affine2'].dw, self.layers['affine2'].db

        return grads
    
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]